<a href="https://colab.research.google.com/github/chaimaerachdi/notebook/blob/main/tp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Chargement du dataset
data = pd.read_csv("movie_review.csv")

# Pré-traitement des données textuelles
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les stopwords
    tokens = word_tokenize(text)
    filtered_text = [word for word in tokens if word not in stop_words]
    return filtered_text

data['text'] = data['text'].apply(preprocess_text)

# Entraînement du modèle Word2Vec
word2vec_model = Word2Vec(sentences=data['text'], vector_size=100, window=5, min_count=1, workers=4)

# Vectorisation des reviews de films
def vectorize_text(text):
    vectors = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

data['Vectorized_Text'] = data['text'].apply(vectorize_text)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data['Vectorized_Text'], data['tag'], test_size=0.2, random_state=42)

# Construction d'un classificateur (Regression Logistique)
classifier = LogisticRegression(max_iter=1000)
classifier.fit(list(X_train), y_train)

# Prédiction sur l'ensemble de test
y_pred = classifier.predict(list(X_test))

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.5715389369592089
Precision: 0.5735235220766333
Recall: 0.5715389369592089
F1 Score: 0.565761658666476
